In [ ]:
class repo_trading:
    
    '''
    Class of function to automate repo trading process.
    
    Note:
    
    You need to import the following packages:
    import pandas as pd
    import warnings
    import pandas as pd
    import numpy as np
    from xbbg import blp
    from datetime import datetime
    import re
    import os
    import subprocess
    from openpyxl import load_workbook
    
    '''
    
    @staticmethod
    def first_short_position(df):
        
        '''
        Return indices (list of tuples) of the first short position in a 5 days horizon.
        
        df (dataframe): must have the following headers "T+0", "T+1", "T+2", "T+3", "T+4".
        
        '''
            
    # Create a matrix with the collateral postions for a 5-day horizon and find the first day a balance becomes negative 
    # for each security

        df_matrix = df[["T+0", "T+1", "T+2", "T+3", "T+4"]]

        negative_indices = []

        for index, row in df_matrix.iterrows():

            for col in df_matrix.columns:

                if row[col] < 0:

                    negative_indices.append((index, col))

                    break

        return negative_indices
    
    @staticmethod
    def close_repo(collateral, financing):
        
        '''
        Return a dataframe with all the funding trades to close and reopen to cover short bond positions.
        
        collateral (dataframe): df from aim_manipulation.collateral_positions()
        financing (dataframe): df from aim_manipulation.financing_trade_blotter()
        
        '''
        
        short_positions = repo_trading.first_short_position(collateral) 
        
        df_short = collateral.loc[[pos[0] for pos in short_positions], # extract index from the rows that have a negative balance
                                
                                ["Account", "Curr", "Security", "ISIN"]].assign( # specify columns to select
            
            Position = [collateral.loc[pos[0], pos[1]] for pos in short_positions], 
            
            Settlement = [pos[1] for pos in short_positions]).sort_values(by = ["Account", "Security"],
                                                                         
                                                                         ascending = [True, True]).reset_index(drop = True)
        
        instructions = []
        
        for i, short in df_short.iterrows():
            
            df_temp = financing[(financing["Identifier"] == short["ISIN"]) & 
                               
                               (financing["Account"] == short["Account"])].sort_values(by = ["Rate", "Quantity"], 
                                                                                          
                                                                                          ascending = [False, True]).reset_index(drop = True)
            
            position_to_cover = short["Position"]
            
            for j, repo in df_temp.iterrows():
                
                if position_to_cover > 0:
                    
                    break
                    
                if  position_to_cover + repo["Quantity"] > 0:
                
                    to_reopen = position_to_cover + repo["Quantity"]
                    
                else:
                    
                    to_reopen = 0
                    
                instructions.append({
                    
                    "Account" : repo["Account"],
                    
                    "Collateral" : repo["Collateral"],
                    
                    "Identifier" : repo["Identifier"],
                    
                    "Close" : repo["Quantity"],
                    
                    "Open" : to_reopen,
                    
                    "Currency" : repo["Curr"],
                    
                    "Settlement": short["Settlement"],
                    
                    "Rate" : repo["Rate"],
                    
                    "Broker" : repo["Broker"]                    
                    
                })
                
                position_to_cover += repo["Quantity"]
                
        df_instructions = pd.DataFrame(instructions)
    
    @staticmethod
    def open_repo(collateral, cash, curr = ["EUR", "USD", "GBP", "AUD"]):
        
        '''
        Return a dataframe with suggested bonds to fund to cover short cash positions.
        
        collateral (dataframe): df from aim_manipulation.collateral_positions()
        cash (dataframe): df from aim_manipulation.cash_ladder()
        
        '''
        
        is_levered = {"ACACIA" : "Levered", "ARROLS" : "Levered", "BSHBRD" : "Unlevered", 

                       "CASBFL" : "Unlevered", "CASBFZ" : "Levered", "CCCSOP" : "Levered",

                       "CWBF" : "Unlevered","CRCCCF" : "Unlevered", "EMLDLS" : "Levered", 

                       "FIXD"   : "Levered", "GESHBI" : "Unlevered", "HBFAC" : "Unlevered", 

                       "HBFHI" : "Unlevered", "IACF" : "Unlevered", "INOMGB" : "Unlevered",

                       "KHAKI" : "Levered",  "LEDLS2" : "Levered", "LGIAHB" : "Levered", 

                       "LSCF" : "Levered", "MOON" : "Levered", "PASKHI" : "Levered", 

                       "PCGA" : "Unlevered", "PETSURE" : "Unlevered", "RAERICH": "Levered", 

                       "SBCF" : "Levered", "SBRK" : "Levered", "SCPIF"  : "Levered", "SMAC" : "Unlevered", 

                       "SMHI" : "Unlevered", "SWANHI" : "Unlevered", "USEC2A" : "Unlevered", 

                       "USEC2D" : "Unlevered", "USECOL" : "Unlevered", "YLD" : "Levered"}
               
        cash_ladder = cash
        
        cash_ladder["Account + Currency"] = cash_ladder["Account"] + " " + cash_ladder["Currency"]
        
        cash_ladder = cash_ladder.merge(pd.DataFrame([is_levered]).transpose().reset_index().rename(columns = {"index" : "Account", 0 : "Account Type"}), on = "Account")
        
        def is_weekend(date_str):
    
            try:

                date_obj = datetime.strptime(date_str, "%m/%d/%y")
        
                return date_obj.weekday() in [5, 6]
    
            except ValueError:
            
                return False
        
        date_pattern = r'\d{1,2}/\d{1,2}/\d{2}'
        
        today = datetime.today() 

        formatted_date = today.strftime("%m/%d/%y")

        date1 = datetime.strptime(formatted_date, "%m/%d/%y")

        date2 = datetime.strptime(cash_ladder.columns[2], "%m/%d/%y")

        date_difference = date2 - date1

        number_of_days = date_difference.days
                    
        for number, column in enumerate(cash_ladder.columns):
    
            if re.match(date_pattern, column):
        
                if is_weekend(column) == True:
            
                    cash_ladder = cash_ladder.drop(column, axis = 1)

        for number, column in enumerate(cash_ladder.columns):
    
            if re.match(date_pattern, column):
    
                if pd.to_datetime(column, format='%m/%d/%y').strftime('%m/%d/%y') == formatted_date:
        
                    cash_ladder.rename(columns = {cash_ladder.columns[number] : "T+" + str(0), 
                                      
                                                  cash_ladder.columns[(number + 1)] : "T+" + str(1),

                                                  cash_ladder.columns[(number + 2)] : "T+" + str(2),
                              
                                                  cash_ladder.columns[(number + 3)] : "T+" + str(3),
                              
                                                  cash_ladder.columns[(number + 4)] : "T+" + str(4)}, 
                           
                                       inplace = True) 
        
                    break
               
        cash_ladder = cash_ladder[["Account + Currency", "Account", "Currency", "Account Type", "T+0", "T+1", "T+2", "T+3", "T+4"]]
        
        cash_ladder = cash_ladder[cash_ladder["Account Type"] == "Levered"].reset_index(drop = True)
        
        short_positions = repo_trading.first_short_position(cash_ladder)
        
        df_short = cash_ladder.loc[[pos[0] for pos in short_positions], # extract index from the rows that have a negative balance
                                
                                ["Account", "Currency", "Account + Currency", "Account Type"]].assign( # specify columns to select
            
            Position = [cash_ladder.loc[pos[0], pos[1]] for pos in short_positions], 
            
            Settlement = [pos[1] for pos in short_positions]).sort_values(by = ["Account", "Currency"],
                                                                         
                                                                         ascending = [True, True]).reset_index(drop = True)
               
        xfin = collateral

        fields = ["payment_rank", "px_last", "min_piece", "min_increment"] # fields to perform bbg query

        xfin = xfin[(xfin["Curr"].notna()) & (xfin["Sect"].isin(["Corp", "US DOMESTIC"]))].reset_index(drop = True) # filter bonds we can lend

        xfin["ISIN"] = xfin["ISIN"] + " Corp" # format bonds so that we can use blp

        bdp = blp.bdp(xfin["ISIN"].unique(), fields)

        bdp.reset_index(inplace = True)

        bdp.rename(columns = {"index" : "ISIN"}, inplace = True)

        xfin = xfin.merge(bdp, on = "ISIN")

        xfin.rename(columns = {xfin.columns[8] : "Net Position T+0", 
                               
                               xfin.columns[9] : "Net Position T+1", 
                       
                               xfin.columns[10] : "Net Position T+2", 
                       
                               xfin.columns[11] : "Net Position T+3", 
                       
                               xfin.columns[12] : "Net Position T+4"}, 
            
                    inplace = True)

        xfin["Account + Currency"] = xfin["Account"] + " " + xfin["Curr"]
        
        for number in range(0, 5):
    
            xfin["Mkt Val T+" + str(number)] = xfin["Net Position T+" + str(number)] * xfin["px_last"] * 10 # get Cash Price from Net Position                   
        
        assumed_haircut = 0.08
               
        instructions = {}
        
        for i, short in df_short.iterrows():
            
            instructions[short["Account + Currency"]] = {}
            
            sorted_collateral = xfin[(xfin["Account + Currency"] == short["Account + Currency"]) & 
                                
                                     (xfin["Net Position " + short["Settlement"]] > 0) & 
                                    
                                     (xfin["payment_rank"] != "Subordinated")].sort_values(by = "Net Position " + short["Settlement"], 
                                                                                                
                                                                                        ascending = False).reset_index(drop = True)
            
            sorted_collateral["Loan Amount"] = sorted_collateral["Mkt Val " + short["Settlement"]] * (1 - assumed_haircut)
            
            cash = short["Position"]
            sum = 0
            collateral = []
            isin = []
            amount = []
            loan_amount = []
            remaining_cash = []
            
            for j, security in enumerate(sorted_collateral["Security"]):
                
                if cash > 0:
                    
                    break
                    
                sum += (sorted_collateral["Loan Amount"][j])
                
                cash += (sorted_collateral["Loan Amount"][j])
                
                collateral.append(security)
                
                isin.append(sorted_collateral["ISIN"][j])
                
                amount.append(sorted_collateral["Net Position " + short["Settlement"]][j])
                
                loan_amount.append(sorted_collateral["Loan Amount"][j])
                
                remaining_cash.append(cash)
            
            instructions[short["Account + Currency"]]["Security"] = collateral
            
            instructions[short["Account + Currency"]]["Isin"] = isin
            
            instructions[short["Account + Currency"]]["Amount"] = amount
            
            instructions[short["Account + Currency"]]["Loan Amount"] = loan_amount
            
            instructions[short["Account + Currency"]]["Remaining Cash"] = remaining_cash
                        
        inx = []
        
        for account_currency, info in instructions.items():
            
            num_entries = len(info['Security'])  # Number of securities per account
            
            for i in range(num_entries):
                
                inx.append([account_currency,
                                        
                                        info['Security'][i],
                                        
                                        info['Isin'][i],
                                        
                                        info['Amount'][i],
                                        
                                        info['Loan Amount'][i],
                                        
                                        info['Remaining Cash'][i]])

        return pd.DataFrame(inx, columns=["Account", "Security", "Isin", "Amount", "Loan Amount", "Remaining Cash"])
            
